In [ ]:
!apt install tesseract-ocr
!pip install pytesseract opencv-python


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 4 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (8,061 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 119600 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e

CLASIFICACIÓN DE LAS IMÁGENES DEL DATASET SEGÚN SI CONTIENEN TEXTO O NO

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

# Directorio raíz que contiene los subdirectorios con imágenes
directorio_raiz = '/content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset'
# Archivo csv destino
csv_path = '/content/drive/My Drive/TFG PABLO VILLALBA BRISA/Archivos/insta_posts.csv'

Mounted at /content/drive


VERSIÓN FINAL CON OPTIMIZACIÓN DE RECURSOS

In [ ]:
import os
import pandas as pd
import cv2
import pytesseract
import gc

#Función para extraer shortcode
def extraer_shortcode(nombre_imagen):
    if nombre_imagen.endswith("UTC.jpg"):
        return nombre_imagen.replace(".jpg", "")
    elif nombre_imagen.endswith("UTC_1.jpg"):
        return nombre_imagen.replace("_1.jpg", "")
    return None

#Función para detectar texto en imagen
def detect_text(image_path):
    #Carga la imagen con OpenCV
    gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    #Reconocimiento de texto
    text = pytesseract.image_to_string(gray)

    #¿Texto?
    if text and len(text.strip()) > 0:
        return "SÍ"
    else:
        return "NO"

#Función para detectar texto en una carpeta de imágenes
def detect_text_in_folder(folder_path, batch_size=10):
    # Lista para almacenar los resultados de texto detectado para cada imagen
    results = []
    image_paths = []
    shortcodes = []

    #Recorrer archivos directorio
    for root, _, files in os.walk(folder_path):
        for nombre_imagen in files:
            #Verificar si es imagen
            if nombre_imagen.endswith('.jpg') or nombre_imagen.endswith('.png') or nombre_imagen.endswith('.jpeg'):
                #Extraer shortcode
                shortcode = extraer_shortcode(nombre_imagen)
                if not shortcode:
                    continue

                #Ruta imagen
                image_path = os.path.join(root, nombre_imagen)
                image_paths.append(image_path)
                shortcodes.append(shortcode)

                #Procesar por lotes
                if len(image_paths) == batch_size:
                    #Detectar texto lote
                    for img_path, sc in zip(image_paths, shortcodes):
                        result = detect_text(img_path)
                        results.append((sc, result))

                    #Limpiar listas
                    image_paths = []
                    shortcodes = []

                    #Liberar memoria
                    gc.collect()

    #Procesar cualquier imagen restante
    if image_paths:
        for img_path, sc in zip(image_paths, shortcodes):
            result = detect_text(img_path)
            results.append((sc, result))

        #Liberar memoria
        gc.collect()

    return results

#Diccionario res
text_results_dict = {'post': [], 'Texto Detectado': []}

#Detectar texto en imágenes de carpeta
text_results = detect_text_in_folder(directorio_raiz)

#Agregar resultados al diccionario
for shortcode, text_detected in text_results:
    text_results_dict['post'].append(shortcode)
    text_results_dict['Texto Detectado'].append(text_detected)

df_text = pd.DataFrame(text_results_dict)

df = pd.read_csv(csv_path)

df_actualizado = pd.merge(df, df_text, on="post", how="left")

df_actualizado.to_csv(csv_path, index=False)


Nueva versión para generar Csv independiente con las predicciones

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

#Subdirectorios
directorio_raiz = '/content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset'
#Archivo CSV independiente
nuevo_csv_path = '/content/drive/My Drive/TFG PABLO VILLALBA BRISA/Archivos/text_predictions.csv'

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import cv2
import pytesseract
import gc

#Función extraer el shortcode
def extraer_shortcode(nombre_imagen):
    if nombre_imagen.endswith("UTC.jpg"):
        return nombre_imagen.replace(".jpg", "")
    elif nombre_imagen.endswith("UTC_1.jpg"):
        return nombre_imagen.replace("_1.jpg", "")
    return None

#Función detectar texto imagen
def detect_text(image_path):
    #Carga imagen con OpenCV
    gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    #Aplica rec. texto
    text = pytesseract.image_to_string(gray)

    #¿Texto?
    if text and len(text.strip()) > 0:
        return "SÍ"
    else:
        return "NO"

#Función detectar texto en carpeta imágenes (llama a detect_text cuando corresponde)
def detect_text_in_folder(folder_path, batch_size=10):
    #Lista res
    results = []
    image_paths = []
    shortcodes = []
    usernames = []

    #Recorrer archivos carpeta
    for root, _, files in os.walk(folder_path):
        for nombre_imagen in files:
            #Comprobar si imagen
            if nombre_imagen.endswith("UTC.jpg") or nombre_imagen.endswith("UTC_1.jpg"):
                #Extraer shortcode
                shortcode = extraer_shortcode(nombre_imagen)
                if not shortcode:
                    continue

                #Ruta imagen
                image_path = os.path.join(root, nombre_imagen)
                image_paths.append(image_path)
                shortcodes.append(shortcode)
                usuario = os.path.basename(root)
                usernames.append(usuario)

                #Procesar por lotes
                if len(image_paths) == batch_size:
                    #Detectar texto lote imágenes
                    for img_path, sc, usuario in zip(image_paths, shortcodes, usernames):
                        result = detect_text(img_path)
                        results.append((usuario, sc, result))

                    #Limpiar las listas
                    image_paths = []
                    shortcodes = []
                    usernames = []

                    #Liberar memoria
                    gc.collect()

    #Procesar imags restantes
    if image_paths:
        for img_path, sc, usuario in zip(image_paths, shortcodes, usernames):
            result = detect_text(img_path)
            results.append((usuario, sc, result))

        #Liberar memoria
        gc.collect()

    return results

#Diccionario res
text_results_dict = {'username': [], 'post': [], 'texto detectado': []}

#Detectar texto en imgs carpeta
text_results = detect_text_in_folder(directorio_raiz)

#Res al diccionario
for usuario, shortcode, text_detected in text_results:
    text_results_dict['username'].append(usuario)
    text_results_dict['post'].append(shortcode)
    text_results_dict['texto detectado'].append(text_detected)

df_text = pd.DataFrame(text_results_dict)

df_text.to_csv(nuevo_csv_path, index=False)
